# Imports

In [1]:
import torch 
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt

# Device

In [2]:
device = 'GPU' if torch.cuda.is_available() else 'cpu'

# Hyper-Parameters

In [3]:
input_channels = 1
num_classes = 10
batch_size = 64
num_epoch = 2
learning_rate = 0.001

# Data

In [4]:
training_data = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
testing_data = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)

train_loader = torch.utils.data.DataLoader(dataset=training_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=testing_data, batch_size=batch_size, shuffle=True) 
# i thik we've 10000 test samples


# CNN

In [5]:
class CNN(nn.Module):
    
    def __init__(self, input_channels=1, num_classes=10):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1)) 
        
        self.fc1 = nn.Linear(in_features=16*7*7, out_features=num_classes)
        
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x

model = CNN().to(device=device)
model

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)

# Loss and optim

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Trianing loop

In [9]:
for epoch in range(num_epoch):
    corr = 0
    n_samples = 0
    ix = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        score_pred = model(images)
        
        loss = criterion(score_pred, labels)
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()
        
        ix += 1
        
        if ix % 200 == 0:
            print(f"Loss_error: {loss}")
            
            
    #Check accu
    _, predictions = score_pred.max(1)
    corr += (predictions == labels).sum()
    n_samples += predictions.size(0)       
    print(f"Accuracey >> : {float(corr) / float(n_samples) * 100}")
    
       
        
        

Loss_error: 0.04050517454743385
Loss_error: 0.06576967984437943
Loss_error: 0.094292052090168
Loss_error: 0.09137357026338577
Accuracey >> : 93.75
Loss_error: 0.10278960317373276
Loss_error: 0.12458086758852005
Loss_error: 0.08423210680484772
Loss_error: 0.0528809018433094
Accuracey >> : 100.0


# Testing 

In [10]:
with torch.no_grad():
    
    corr_pred = 0
    num_samples = 0
    
    for x, y in test_loader:
        x = x.to(device)
        y = y.to(device)
        
        scores = model(x)
        
        _, predictions = scores.max(1)
        corr_pred += (predictions == y).sum()
        num_samples += predictions.size(0)
                
    print(f"Accuracey >> : {float(corr_pred) / float(num_samples) * 100}")
    

         

Accuracey >> : 98.52


# Renato